In [1]:
import pandas as pd
import numpy as np
import pycountry as pc

In [2]:
source_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/'
files = [
    'time_series_covid19_confirmed_global.csv',
    'time_series_covid19_deaths_global.csv',
    'time_series_covid19_recovered_global.csv'
]

In [3]:
# fetching csv files from github

dfs_raw = [pd.read_csv(source_url + f) for f in files]
dfs_raw[0].head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,1092,1176,1279,1351,1463,1531,1703,1828,1939,2171
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,609,634,663,678,712,726,736,750,766,773
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,2811,2910,3007,3127,3256,3382,3517,3649,3848,4006
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,717,723,723,731,738,738,743,743,743,745
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,24,25,25,25,25,26,27,27,27,27


In [4]:
# deleting unnecessary columns

drop_columns = ['Province/State', 'Lat', 'Long']
dfs_dropped = [df.drop(drop_columns, axis=1) for df in dfs_raw]

sample = dfs_dropped[0]
sample[sample['Country/Region'] == 'Australia']

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20
8,Australia,0,0,0,0,0,0,0,0,0,...,104,104,104,105,106,106,106,106,106,106
9,Australia,0,0,0,0,3,4,4,4,4,...,2969,2971,2976,2982,2994,3002,3004,3016,3016,3025
10,Australia,0,0,0,0,0,0,0,0,0,...,28,28,28,28,28,28,28,28,28,28
11,Australia,0,0,0,0,0,0,0,1,3,...,1024,1024,1026,1026,1026,1030,1033,1034,1033,1033
12,Australia,0,0,0,0,0,0,0,0,0,...,437,438,438,438,438,438,438,438,438,438
13,Australia,0,0,0,0,0,0,0,0,0,...,201,205,207,207,207,212,214,218,219,221
14,Australia,0,0,0,0,1,1,1,1,2,...,1336,1336,1337,1343,1346,1349,1349,1354,1361,1364
15,Australia,0,0,0,0,0,0,0,0,0,...,546,546,546,548,549,549,549,550,551,551


In [5]:
%%time

# converting countries names to alpha 2 codes

manual_coding = {
    'Congo (Brazzaville)': 'CD',
    'Congo (Kinshasa)': 'CD',
    'Korea, South': 'KR',
    'Taiwan*': 'TW',
    'Laos': 'LA',
    'Burma': 'MM',
}

missed_names = []

def get_country_code(name):
    try:
        country_data = pc.countries.search_fuzzy(name)        
        return country_data[0].alpha_2
    except LookupError:
        manual_lookup = manual_coding.get(name)
        if manual_lookup is None:
            missed_names.append(name)
            
        return manual_lookup

def add_codes_column(df):
    codes = [get_country_code(record['Country/Region']) for _, record in df.iterrows()]
    df['Alpha2'] = codes
    return df

dfs_with_codes = [add_codes_column(df) for df in dfs_dropped]
dfs_with_codes = [df.drop(['Country/Region'], axis=1) for df in dfs_with_codes]

print(f'missed names: {missed_names}')
dfs_with_codes[0].head(10)

missed names: ['Diamond Princess', 'West Bank and Gaza', 'MS Zaandam', 'Diamond Princess', 'West Bank and Gaza', 'MS Zaandam', 'Diamond Princess', 'West Bank and Gaza', 'MS Zaandam']
CPU times: user 37.4 s, sys: 19.5 ms, total: 37.4 s
Wall time: 37.4 s


,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,Alpha2
0,0,0,0,0,0,0,0,0,0,0,...,1176,1279,1351,1463,1531,1703,1828,1939,2171,AF
1,0,0,0,0,0,0,0,0,0,0,...,634,663,678,712,726,736,750,766,773,AL
2,0,0,0,0,0,0,0,0,0,0,...,2910,3007,3127,3256,3382,3517,3649,3848,4006,DZ
3,0,0,0,0,0,0,0,0,0,0,...,723,723,731,738,738,743,743,743,745,AD
4,0,0,0,0,0,0,0,0,0,0,...,25,25,25,25,26,27,27,27,27,AO
5,0,0,0,0,0,0,0,0,0,0,...,24,24,24,24,24,24,24,24,24,AG
6,0,0,0,0,0,0,0,0,0,0,...,3144,3435,3607,3780,3892,4003,4127,4285,4428,AR
7,0,0,0,0,0,0,0,0,0,0,...,1473,1523,1596,1677,1746,1808,1867,1932,2066,AM
8,0,0,0,0,0,0,0,0,0,0,...,104,104,105,106,106,106,106,106,106,AU
9,0,0,0,0,3,4,4,4,4,4,...,2971,2976,2982,2994,3002,3004,3016,3016,3025,AU


In [6]:
# grouping records by country code

dfs_grouped = [df.groupby(['Alpha2'], as_index=False).sum() for df in dfs_with_codes]
dfs_grouped[0].head()

,Alpha2,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20
0,AD,0,0,0,0,0,0,0,0,0,...,717,723,723,731,738,738,743,743,743,745
1,AE,0,0,0,0,0,0,0,4,4,...,7755,8238,8756,9281,9813,10349,10839,11380,11929,12481
2,AF,0,0,0,0,0,0,0,0,0,...,1092,1176,1279,1351,1463,1531,1703,1828,1939,2171
3,AG,0,0,0,0,0,0,0,0,0,...,23,24,24,24,24,24,24,24,24,24
4,AL,0,0,0,0,0,0,0,0,0,...,609,634,663,678,712,726,736,750,766,773


In [7]:
# saving clean files

dest_dir = './who-data/'

for df, file in zip(dfs_grouped, files):
    df.to_csv(f'{dest_dir}/CLEAN-{file}', index=False, header=True)